# Análisis de rendimientos (variaciones) de ADRs

## Parametrizado en días

In [12]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# PARÁMETROS DE ENTRADA
DIAS_ANALISIS = 30  # Período de análisis en días (ej: 30, 90, 180, 365)

# ADRs Argentinos
ADRS_ARGENTINOS = {
    'BBAR': 'BBAR',
    'BMA': 'BMA',
    'CEPU': 'CEPU',
    'CRESY': 'CRESY',
    'EDN': 'EDN',
    'GGAL': 'GGAL',
    'IRS': 'IRS',
    'LOMA': 'LOMA',
    'PAM': 'PAM',
    'SUPV': 'SUPV',
    'TEO': 'TEO',
    'TGS': 'TGS',
    'YPF': 'YPF'
}

# ============================================
# FUNCIONES
# ============================================

def formato_argentino(numero):
    # Formatear con 2 decimales y coma
    numero_str = f"{numero:,.2f}"
    # Intercambiar punto y coma para formato argentino
    numero_str = numero_str.replace(',', 'TEMP').replace('.', ',').replace('TEMP', '.')
    return numero_str


def obtener_datos_acciones(dias):
    fecha_fin = datetime.now()
    fecha_inicio = fecha_fin - timedelta(days=dias)
    
    print(f"Obteniendo datos desde {fecha_inicio.strftime('%d/%m/%Y')} hasta {fecha_fin.strftime('%d/%m/%Y')}...")
    print("=" * 80)
    
    resultados = []
    
    for nombre, ticker in ADRS_ARGENTINOS.items():
        try:
            # Descargo datos sin auto_adjust para tener precios reales
            data = yf.download(ticker, start=fecha_inicio, end=fecha_fin, progress=False, auto_adjust=False)
            
            if len(data) < 2:
                print(f"⚠️  {nombre}: Sin datos suficientes")
                continue
            
            # Uso Close directo (precio real sin ajustes)
            precio_inicial = float(data['Close'].iloc[0])
            precio_final = float(data['Close'].iloc[-1])
            
            # Calculo variación porcentual
            variacion_pct = ((precio_final - precio_inicial) / precio_inicial) * 100
            
            resultados.append({
                'Acción': nombre,
                'Ticker': ticker,
                'Precio Inicial USD': precio_inicial,
                'Precio Final USD': precio_final,
                'Variación %': variacion_pct
            })
            
            print(f"✓ {nombre}: USD {precio_inicial:.2f} → USD {precio_final:.2f} ({variacion_pct:+.2f}%)")
            
        except Exception as e:
            print(f"✗ Error con {nombre}: {str(e)}")
    
    return pd.DataFrame(resultados)


def analizar_rendimientos(df):
    if df.empty:
        print("\n❌ No se pudieron obtener datos de las acciones")
        return
    
    # Ordeno por variación
    df_sorted = df.sort_values('Variación %', ascending=False).reset_index(drop=True)
    
    # Separo acciones positivas y negativas
    df_positivas = df_sorted[df_sorted['Variación %'] > 0]
    df_negativas = df_sorted[df_sorted['Variación %'] < 0]
    
    print("\n" + "=" * 90)
    print("📊 ANÁLISIS DE RENDIMIENTO - ADRs ARGENTINOS")
    print("=" * 90)
    
    # Muestro top que más subieron
    if len(df_positivas) > 0:
        num_top = min(5, len(df_positivas))
        print(f"\n🚀 TOP {num_top} ADRs CON MEJOR RENDIMIENTO:")
        print("-" * 90)
        print(f"{'TICKER':<12} {'PRECIO INICIAL':>18} {'PRECIO FINAL':>18} {'VARIACIÓN':>15}")
        print("-" * 90)
        
        for idx, row in df_positivas.head(num_top).iterrows():
            precio_inicial_fmt = formato_argentino(row['Precio Inicial USD'])
            precio_final_fmt = formato_argentino(row['Precio Final USD'])
            print(f"{row['Acción']:<12} {'USD ' + precio_inicial_fmt:>18} {'USD ' + precio_final_fmt:>18} {row['Variación %']:>13.2f}% 📈")
    
    # Solo muestro las que bajaron si realmente hay
    if len(df_negativas) > 0:
        num_bottom = min(5, len(df_negativas))
        print(f"\n📉 TOP {num_bottom} ADRs CON PEOR RENDIMIENTO:")
        print("-" * 90)
        print(f"{'TICKER':<12} {'PRECIO INICIAL':>18} {'PRECIO FINAL':>18} {'VARIACIÓN':>15}")
        print("-" * 90)
        
        for idx, row in df_negativas.head(num_bottom).iterrows():
            precio_inicial_fmt = formato_argentino(row['Precio Inicial USD'])
            precio_final_fmt = formato_argentino(row['Precio Final USD'])
            print(f"{row['Acción']:<12} {'USD ' + precio_inicial_fmt:>18} {'USD ' + precio_final_fmt:>18} {row['Variación %']:>13.2f}% 📉")
    else:
        print("\n✨ ¡TODAS LAS ADRs CERRARON EN POSITIVO!")
        print("-" * 90)
    
    # Estadísticas generales con mejor formato
    print("\n" + "=" * 90)
    print("📈 RESUMEN GENERAL DEL PERÍODO")
    print("=" * 90)
    
    promedio = df['Variación %'].mean()
    mediana = df['Variación %'].median()
    mejor = df_sorted.iloc[0]
    
    # Emoji según el promedio
    emoji_promedio = "🟢" if promedio > 0 else "🔴" if promedio < 0 else "⚪"
    
    print(f"\n{'Rendimiento Promedio:':<30} {emoji_promedio} {promedio:>7.2f}%")
    print(f"{'Rendimiento Mediano:':<30} {mediana:>10.2f}%")
    print(f"{'Total ADRs Analizados:':<30} {len(df):>10}")
    print(f"{'ADRs con Rendimiento Positivo:':<30} {len(df_positivas):>10} ({len(df_positivas)/len(df)*100:.1f}%)")
    print(f"{'ADRs con Rendimiento Negativo:':<30} {len(df_negativas):>10} ({len(df_negativas)/len(df)*100:.1f}%)")
    
    print(f"\n🏆 MEJOR PERFORMANCE: {mejor['Acción']} con {mejor['Variación %']:+.2f}%")
    
    if len(df_negativas) > 0:
        peor = df_sorted.iloc[-1]
        print(f"⚠️  PEOR PERFORMANCE: {peor['Acción']} con {peor['Variación %']:+.2f}%")
    
    print("\n" + "=" * 90)
    
    return df_sorted


# ============================================
# EJECUCIÓN PRINCIPAL
# ============================================

if __name__ == "__main__":
    print("\n" + "=" * 90)
    print("🇦🇷  ANÁLISIS DE ADRs ARGENTINOS  🇺🇸")
    print(f"📅 Período: {DIAS_ANALISIS} días ({(DIAS_ANALISIS/30):.1f} meses aprox.)")
    print("=" * 90 + "\n")
    
    # Obtengo datos
    df_acciones = obtener_datos_acciones(DIAS_ANALISIS)
    
    # Analizar y mostrar resultados
    if not df_acciones.empty:
        df_resultados = analizar_rendimientos(df_acciones)
        
        # Guardar resultados en CSV
        archivo_salida = f'analisis_adrs_{DIAS_ANALISIS}dias_{datetime.now().strftime("%Y%m%d")}.csv'
        df_resultados.to_csv(archivo_salida, index=False)
        print(f"\n💾 Resultados guardados en: {archivo_salida}")
    else:
        print("\n⚠️  No se pudieron obtener datos suficientes para el análisis")
    
    print("\n✅ Análisis completado\n")


🇦🇷  ANÁLISIS DE ADRs ARGENTINOS  🇺🇸
📅 Período: 30 días (1.0 meses aprox.)

Obteniendo datos desde 19/12/2025 hasta 18/01/2026...
✓ BBAR: USD 18.32 → USD 17.68 (-3.49%)
✓ BMA: USD 90.84 → USD 92.67 (+2.01%)
✓ CEPU: USD 17.48 → USD 15.29 (-12.53%)
✓ CRESY: USD 11.61 → USD 12.37 (+6.55%)
✓ EDN: USD 31.36 → USD 27.08 (-13.65%)
✓ GGAL: USD 53.72 → USD 51.39 (-4.34%)
✓ IRS: USD 15.35 → USD 16.79 (+9.38%)
✓ LOMA: USD 12.80 → USD 11.71 (-8.52%)
✓ PAM: USD 88.51 → USD 79.68 (-9.98%)
✓ SUPV: USD 11.91 → USD 10.71 (-10.08%)
✓ TEO: USD 11.24 → USD 10.93 (-2.76%)
✓ TGS: USD 31.05 → USD 28.88 (-6.99%)
✓ YPF: USD 35.51 → USD 34.31 (-3.38%)

📊 ANÁLISIS DE RENDIMIENTO - ADRs ARGENTINOS

🚀 TOP 3 ADRs CON MEJOR RENDIMIENTO:
------------------------------------------------------------------------------------------
TICKER           PRECIO INICIAL       PRECIO FINAL       VARIACIÓN
------------------------------------------------------------------------------------------
IRS                   USD 15,35    